In [3]:
!python3 -m pip install --force-reinstall conllu

Defaulting to user installation because normal site-packages is not writeable
  Using cached conllu-4.5.3-py2.py3-none-any.whl (16 kB)
  Attempting uninstall: conllu
    Found existing installation: conllu 4.5.3
    Uninstalling conllu-4.5.3:
      Successfully uninstalled conllu-4.5.3


In [1]:
import pandas as pd
from conllu import parse
from torch.utils.data import Dataset, DataLoader

/home/afz225/.conda/envs/fairseq/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
msa_file="../conllu_analysis/data/PATB/catib.train.v2.11062020.conllu"
da_file="../conllu_analysis/data/dialect_data/Levantine/lev_train.conllx"

In [3]:
msa_fp = open(msa_file, 'r')
da_fp = open(da_file, 'r')
msa_data = "".join(msa_fp.readlines())
da_data = "".join(da_fp.readlines())
da_fp.close()
msa_fp.close()

In [4]:
msa_sents = parse(msa_data)
da_sents = parse(da_data)

In [5]:
table_headers = ['token_id', 'sentence', 'UPOS-deprel']
msa_list = []
da_list = []

In [6]:
for sent in msa_sents:
    sent_string = " ".join([tok['form'] for tok in sent])
    for tok in sent:
        msa_list.append([tok['id'], sent_string, tok['upos']+'+'+tok['deprel']])

In [7]:
for sent in da_sents:
    sent_string = " ".join([tok['form'] for tok in sent])
    for tok in sent:
        da_list.append([tok['id'], sent_string, tok['upos']+'+'+tok['deprel']])

In [8]:
msa_df = pd.DataFrame(msa_list, columns=table_headers)

In [9]:
da_df = pd.DataFrame(da_list, columns=table_headers)

In [10]:
def random_pairing(df1, df2):
    paired_dfs = pd.DataFrame()
    for gram_cat in df1['UPOS-deprel'].unique():
        num_elems = df1['UPOS-deprel'].value_counts().loc[gram_cat]
        matches = df2[df2['UPOS-deprel'] == gram_cat]
        if len(matches) != 0:
            pairings = matches.sample(n=num_elems, replace=True).sample(frac=1)
            paired_df = pd.concat([df1[df1['UPOS-deprel'] == gram_cat].reset_index(drop=True).drop(['UPOS-deprel'], axis=1), pairings.reset_index(drop=True).drop(['UPOS-deprel'], axis=1)], axis=1)
            paired_dfs = pd.concat([paired_dfs, paired_df], axis=0)
        return paired_dfs
        

In [16]:
random_pairing(msa_df, da_df).to_csv("msa_aligned_lev_paired_tokens.tsv", header=None, index=None,sep='\t')

In [13]:
df = pd.read_csv("lev_aligned_msa_paired_tokens.tsv", header=None, sep='\t')

In [14]:
df.columns = ['reference_token_id', 'reference_sent', 'aligning_token_id', 'aligning_sent']

In [15]:
df

,reference_token_id,reference_sent,aligning_token_id,aligning_sent
0,1,ألو,2,"ذ الرجال : الجزيرة ( مصر ) , الاتحاد السكندري ..."
1,1,ألو,1,ثانيا : إجراء تحقيقات موسعة ب+ النسبة إلى أسبا...
2,1,ألو,17,و+ يمكن تلقيم النصوص على شبكة الإنترنت و+ قراء...
3,1,أيوه,93,وزعت شعبة العلاقات العامة في قوى الأمن الداخلي...
4,1,إيه ريم,2,"ربما 8 "" !"
...,...,...,...,...
2515,1,بس,18,شارك من لبنان الكاردينال موسى داوود ( رئيس مجم...
2516,3,على ال اللي عند ي إل ها عشر دقايق,5,أما التيار الآخر ف+ واقعي و+ يقود +ه حلفاء سور...
2517,5,أنا لأ هاي الجامعة قريبة ب أبو نصير أقرب في ش ...,3,2 - إعداد التشريعات اللازمة ل+ طريقة إدارة هذه...
2518,5,آه إنتوا و الجامعة قريبين,1,الطبعة رقم : 1
